In [ ]:
import os
import glob
import json


In [ ]:
script = '#!/bin/bash\n#PBS -q celltypes\n#PBS -l mem=10g,nodes=1:ppn=10\n#PBS -l walltime=4:00:00\n#PBS -N validation\n#PBS -r n\n#PBS -j oe\n\nsource /etc/profile\nmodule load anaconda\n\nexport TMPDIR="/allen/programs/celltypes/workgroups/em-connectomics/gayathrim/scratch/"\n\n\nexport RENDER_CLIENT_JAR=/allen/aibs/pipeline/image_processing/volume_assembly/render-jars/production/render-ws-java-client-standalone.jar\n\nexport RENDER_JAVA_HOME=/shared/utils.x86_64/jdk_8u91\n\nexport JAVA_HOME=/shared/utils.x86_64/jdk_8u91\n\n\numask 002\n#source activate /allen/aibs/pipeline/image_processing/volume_assembly/conda_envs/render-modules/production\nsource activate /allen/programs/celltypes/workgroups/em-connectomics/danielk/conda/rm_validate_clone\n\nexport PYTHONPATH=/allen/programs/celltypes/workgroups/em-connectomics/gayathrim/pipeline/render-modules\n\npython -m rendermodules.materialize.validate_materialized_tilesource --input_json /allen/programs/celltypes/workgroups/em-connectomics/gayathrim/pipeline/render-modules/ex_validate_materialization.json --minRow {} --maxRow {} --minCol {} --maxCol {} --minZ {} --maxZ {} --output_json /allen/programs/celltypes/workgroups/em-connectomics/gayathrim/scratch/validation/output/chunk_11_validation_{}.json\n'

In [ ]:
rootdir = "/allen/programs/celltypes/production/wijem/workflow_data/production/chunk_11_zs9741_ze10709/17797_1R/FUSEDOUTSTACK/1024x1024/0"
zvalues = range(9741, 10710)

folders = glob.glob(os.path.join(rootdir, "*/*"))
fldrzs = [int(os.path.basename(os.path.dirname(fldr))) for fldr in folders]
subz = [int(os.path.basename(fldr)) for fldr in folders]


fldrlist = {}
for zz in set(fldrzs):
    fldrlist[zz] = {}
    
fldrsetzs = list(set(fldrzs))

for z in fldrsetzs:
    indexes = [i for i, x in enumerate(fldrzs) if z == x]
    vals = [subz[i] for i in indexes]
    fldrlist[z]['min'] = min(vals)
    fldrlist[z]['max'] = max(vals)
    
for z in zvalues:
    if z in fldrsetzs:
        new_script = script.format(fldrlist[z]['min'], fldrlist[z]['max'], fldrlist[z]['min'], fldrlist[z]['max'], z, z, z)
        filename = "/allen/programs/celltypes/workgroups/em-connectomics/gayathrim/scratch/validation/pbs/validate_{}.pbs".format(z)
        with open(filename, "w") as f:
            f.write(new_script)
        f.close()